In [5]:
!pip install dlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 92.0 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dlib: filename=dlib-19.24.2-cp310-cp310-linux_x86_64.whl size=3753609 sha256=22873fbd069e832360f8d5927b324458baf3af3293cb570846f6bd3ccc23d43d
  Stored in directory: /home/jupyter/.cache/pip/wheels/9b/e2/80/888fdc098db86b463ff0c83ae5e5ca151889e901bc1e9a3a11
Successfully built dlib


In [8]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 22.3 MB/s eta 0:00:00:00:0100:01


In [9]:
import tensorflow as tf
import pandas as pd
import numpy as np
from typing import List
import matplotlib.pyplot as plt
#import imageio
import dlib
import cv2
import os
from google.cloud import storage

In [10]:
#!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK
!bunzip2 shape_predictor_68_face_landmarks.dat.bz2
datFile =  "shape_predictor_68_face_landmarks.dat"

bunzip2: Can't open input file shape_predictor_68_face_landmarks.dat.bz2: No such file or directory.


In [11]:
data= pd.read_csv('all_matches.csv')
data

,Unnamed: 0,speaker_align,speaker_vid
0,swwf7p,s34,s34
1,bric4a,s34,s34
2,sgal9p,s34,s34
3,sbax8s,s34,s34
4,bwidza,s34,s34
...,...,...,...
33995,lrwp8s,s23,s22
33996,sgim5n,s23,s22
33997,lwwq5p,s23,s22
33998,lwac9p,s23,s22


In [12]:
df= data[data.speaker_vid.isin(['s1','s2', 's3','s7','s9','s11','s12','s18','s22','s23','s24','s27','s28','s31','s32'])]

In [13]:
random_df= df.sample(2000,random_state=42)
random_df.head(20)

,Unnamed: 0,speaker_align,speaker_vid
25499,bbbx1a,s31,s31
13475,swbbzs,s11,s12
32167,bwbe7p,s24,s24
1862,bbav4s,s33,s32
12970,pbbo3s,s10,s11
13706,brws3n,s11,s12
7017,bbbd1s,s26,s27
7781,pgwi5s,s26,s27
7898,bbbd2p,s26,s27
5250,lwws5a,s3,s3


In [14]:
speaker_names = random_df['speaker_vid'].tolist()


In [15]:
video_names=random_df['Unnamed: 0'].tolist()


In [16]:
ending= '.mpg'
video_names = [item + ending for item in video_names]

In [17]:
video_names

['bbbx1a.mpg',
 'swbbzs.mpg',
 'bwbe7p.mpg',
 'bbav4s.mpg',
 'pbbo3s.mpg',
 'brws3n.mpg',
 'bbbd1s.mpg',
 'pgwi5s.mpg',
 'bbbd2p.mpg',
 'lwws5a.mpg',
 'swbl8s.mpg',
 'lbbi7n.mpg',
 'sral6n.mpg',
 'lraq2a.mpg',
 'pgii9p.mpg',
 'prwv7p.mpg',
 'bwwr3n.mpg',
 'bgbe7a.mpg',
 'lwbj4n.mpg',
 'bral3s.mpg',
 'swwi9a.mpg',
 'bbav9p.mpg',
 'lgwm6s.mpg',
 'bbir8p.mpg',
 'sgaf5n.mpg',
 'prbj3n.mpg',
 'sgagzn.mpg',
 'bwaf3n.mpg',
 'bwwe6s.mpg',
 'pbif1s.mpg',
 'bbbz7n.mpg',
 'bgwb4p.mpg',
 'lgic7a.mpg',
 'prip3a.mpg',
 'bgwn4n.mpg',
 'lwwm1s.mpg',
 'sgiz3n.mpg',
 'lray3n.mpg',
 'bwiq9p.mpg',
 'swah2p.mpg',
 'prav2n.mpg',
 'bgbt3s.mpg',
 'lrbq9a.mpg',
 'prah3n.mpg',
 'lrwp9a.mpg',
 'praa8a.mpg',
 'bwbeza.mpg',
 'sbiq5a.mpg',
 'lwiu5n.mpg',
 'brwk6p.mpg',
 'sbwz8a.mpg',
 'brwa4a.mpg',
 'bbiy8n.mpg',
 'lgakza.mpg',
 'lgwj6n.mpg',
 'bgbn7s.mpg',
 'pwbn5s.mpg',
 'lbbj6n.mpg',
 'sbik9p.mpg',
 'bbip5n.mpg',
 'swaf6p.mpg',
 'sbij8s.mpg',
 'lwac9a.mpg',
 'bbib8n.mpg',
 'lwwp8s.mpg',
 'bgbe5p.mpg',
 'lriu4s.m

In [18]:
combo=zip(speaker_names,video_names)
final= list(combo)

In [19]:
!pwd

/home/jupyter


In [25]:
LIP_MARGIN = 0.4                # Marginal rate for lip-only image.
RESIZE = (70,30)
BASE_URL='raw_data1/'
# VIDEO_PATH=os.path.join(BASE_URL, speaker_names, video_names)
VIDEO_PATH = [os.path.join(BASE_URL, speaker, video) for speaker, video in final]


# Final image size
# for a in range(33):
#     if a != 18:
#         VIDEO_PATH=f'/Users/alessiastroni/code/girishgautam/lip_translate/raw_data/s{a+2}/'


In [26]:
VIDEO_PATH

['raw_data1/s31/bbbx1a.mpg',
 'raw_data1/s12/swbbzs.mpg',
 'raw_data1/s24/bwbe7p.mpg',
 'raw_data1/s32/bbav4s.mpg',
 'raw_data1/s11/pbbo3s.mpg',
 'raw_data1/s12/brws3n.mpg',
 'raw_data1/s27/bbbd1s.mpg',
 'raw_data1/s27/pgwi5s.mpg',
 'raw_data1/s27/bbbd2p.mpg',
 'raw_data1/s3/lwws5a.mpg',
 'raw_data1/s32/swbl8s.mpg',
 'raw_data1/s18/lbbi7n.mpg',
 'raw_data1/s23/sral6n.mpg',
 'raw_data1/s12/lraq2a.mpg',
 'raw_data1/s12/pgii9p.mpg',
 'raw_data1/s12/prwv7p.mpg',
 'raw_data1/s28/bwwr3n.mpg',
 'raw_data1/s31/bgbe7a.mpg',
 'raw_data1/s23/lwbj4n.mpg',
 'raw_data1/s11/bral3s.mpg',
 'raw_data1/s3/swwi9a.mpg',
 'raw_data1/s28/bbav9p.mpg',
 'raw_data1/s2/lgwm6s.mpg',
 'raw_data1/s1/bbir8p.mpg',
 'raw_data1/s32/sgaf5n.mpg',
 'raw_data1/s2/prbj3n.mpg',
 'raw_data1/s27/sgagzn.mpg',
 'raw_data1/s12/bwaf3n.mpg',
 'raw_data1/s28/bwwe6s.mpg',
 'raw_data1/s31/pbif1s.mpg',
 'raw_data1/s2/bbbz7n.mpg',
 'raw_data1/s3/bgwb4p.mpg',
 'raw_data1/s27/lgic7a.mpg',
 'raw_data1/s1/prip3a.mpg',
 'raw_data1/s9/bgwn4n.

In [27]:
len(VIDEO_PATH)

2000

In [28]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [29]:
#video_list = os.listdir(VIDEO_PATH)

# video_list = [path for path in VIDEO_PATH if os.path.isdir(path)]

In [30]:
def shape_to_list(shape):
    coords = []
    for i in range(0, 68):
        coords.append((shape.part(i).x, shape.part(i).y))
    return coords

In [31]:
def makeitgray(image_pls, i, j):
    return np.dot(image_pls[i][j][...,:3], [0.2989, 0.5780, 0.1440])

In [32]:
def standardize(vid):
    mean_vid = vid.mean(axis=0)
    std_vid = vid.std(axis=0)
    return (vid-mean_vid)/std_vid

In [ ]:
cropped_img_list=[]
gray_image_list = []
standardized_list=[]
dict_keys=[]
for video in VIDEO_PATH: # Iterate on video files

    print(video)

    print(f"Processing video: {video}")

    if video.endswith('.mpg'):
        #vid_path = VIDEO_PATH + vid_name
        vid = cv2.VideoCapture(video)

        frames = []               # A list to hold frame images
        frames_colour = []         # A list to hold original frame images
        while(True):
            success, frame = vid.read()
                # Read frame
            if not success:
                break                           # Break if no frame to read left
            gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)   # Convert image into grayscale
            frames.append(gray)                  # Add image to the frame buffer
            frames_colour.append(frame)

        vid.release()

        landmarks = []
        for (i, image) in enumerate(frames):          #iterate on frame lis
            face_rects = detector(image,1)             #detects face
            if len(face_rects) < 1:                 #no faces
                print(f"No face detected: {video}")
                continue
            if len(face_rects) > 1:                  #too many faces
                print(f"Too many faces: {video}")
                continue
            rect = face_rects[0]                    #proper number of faces
            landmark = predictor(image, rect)   #detect face landmarks
            landmark = shape_to_list(landmark)
            landmarks.append(landmark)

        cropped_img = []
        for (i,landmark) in enumerate(landmarks):
            lip_landmark = landmark[48:68]                                          # Landmark corresponding to lip
            lip_x = sorted(lip_landmark,key = lambda pointx: pointx[0])             # Lip landmark sorted for determining lip region
            lip_y = sorted(lip_landmark, key = lambda pointy: pointy[1])
            x_add = int((lip_x[-1][0]-lip_x[0][0])*LIP_MARGIN*1)                     # Determine Margins for lip-only image
            y_add = int((-lip_y[0][1]+lip_y[-1][1])*LIP_MARGIN*2)
            crop_pos = (lip_x[0][0]-x_add, lip_x[-1][0]+x_add, lip_y[0][1]-y_add, lip_y[-1][1]+y_add)
            cropped = frames_colour[i][crop_pos[2]:crop_pos[3],crop_pos[0]:crop_pos[1]]        # Crop image
            cropped = cv2.resize(cropped,(RESIZE[0],RESIZE[1]),interpolation=cv2.INTER_CUBIC)       # Resize
            cropped_img.append(cropped)

        print('------------ images cropped ----------')

        cropped_img_list.append(cropped_img)


        for i, image in enumerate(cropped_img_list):
            gray_frame_list = []
            for j, frame in enumerate(image):
                gray_image = makeitgray(cropped_img_list,i,j)
        # comment this out if you don't need the shape (X, X, 1)
                gray_image = np.expand_dims(gray_image, axis=2)
                gray_frame_list.append(gray_image)
            gray_image_list.append(gray_frame_list)


        for vid in gray_image_list:
    #print(type(vid))
            vid= np.array(vid)
            standard_vid = standardize(vid)
    # standardized_list.append(standard_vid)
            float_vids= standard_vid.astype('float32')
            standardized_list.append(float_vids)

    sample_names= [y for x,y in final]

    for name in sample_names:
        dict_keys.append(name.replace('.mpg', ''))

    final_vids= dict(zip(dict_keys, standardized_list))

np.savez('zipped_vids.npz', **final_vids)

print(f"VIDEO COMPLETED: {video}")


raw_data1/s31/bbbx1a.mpg
Processing video: raw_data1/s31/bbbx1a.mpg
------------ images cropped ----------
raw_data1/s12/swbbzs.mpg
Processing video: raw_data1/s12/swbbzs.mpg
------------ images cropped ----------
raw_data1/s24/bwbe7p.mpg
Processing video: raw_data1/s24/bwbe7p.mpg
------------ images cropped ----------
raw_data1/s32/bbav4s.mpg
Processing video: raw_data1/s32/bbav4s.mpg
------------ images cropped ----------
raw_data1/s11/pbbo3s.mpg
Processing video: raw_data1/s11/pbbo3s.mpg
------------ images cropped ----------
raw_data1/s12/brws3n.mpg
Processing video: raw_data1/s12/brws3n.mpg
------------ images cropped ----------
raw_data1/s27/bbbd1s.mpg
Processing video: raw_data1/s27/bbbd1s.mpg
------------ images cropped ----------
raw_data1/s27/pgwi5s.mpg
Processing video: raw_data1/s27/pgwi5s.mpg
------------ images cropped ----------
raw_data1/s27/bbbd2p.mpg
Processing video: raw_data1/s27/bbbd2p.mpg
------------ images cropped ----------
raw_data1/s3/lwws5a.mpg
Processing vi

/var/tmp/ipykernel_17232/1835718190.py:2: RuntimeWarning: Mean of empty slice.
  mean_vid = vid.mean(axis=0)
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


------------ images cropped ----------
raw_data1/s2/bbbz7n.mpg
Processing video: raw_data1/s2/bbbz7n.mpg
------------ images cropped ----------
raw_data1/s3/bgwb4p.mpg
Processing video: raw_data1/s3/bgwb4p.mpg
------------ images cropped ----------
raw_data1/s27/lgic7a.mpg
Processing video: raw_data1/s27/lgic7a.mpg
------------ images cropped ----------
raw_data1/s1/prip3a.mpg
Processing video: raw_data1/s1/prip3a.mpg
------------ images cropped ----------
raw_data1/s9/bgwn4n.mpg
Processing video: raw_data1/s9/bgwn4n.mpg
------------ images cropped ----------
raw_data1/s1/lwwm1s.mpg
Processing video: raw_data1/s1/lwwm1s.mpg
------------ images cropped ----------
raw_data1/s32/sgiz3n.mpg
Processing video: raw_data1/s32/sgiz3n.mpg
------------ images cropped ----------
raw_data1/s2/lray3n.mpg
Processing video: raw_data1/s2/lray3n.mpg
------------ images cropped ----------
raw_data1/s22/bwiq9p.mpg
Processing video: raw_data1/s22/bwiq9p.mpg
------------ images cropped ----------
raw_data1/

In [125]:
# def makeitgray(image_pls, i, j):
#     return np.dot(image_pls[i][j][...,:3], [0.2989, 0.5780, 0.1440])

In [126]:
# gray_image_list = []
# for i, image in enumerate(cropped_img_list):
#     gray_frame_list = []
#     for j, frame in enumerate(image):
#         gray_image = makeitgray(cropped_img_list,i,j)
#         # comment this out if you don't need the shape (X, X, 1)
#         gray_image = np.expand_dims(gray_image, axis=2)
#         gray_frame_list.append(gray_image)
#     gray_image_list.append(gray_frame_list)

In [127]:
# def standardize(vid):
#     mean_vid = vid.mean(axis=0)
#     std_vid = vid.std(axis=0)
#     return (vid-mean_vid)/std_vid

In [149]:
# standardized_list=[]
# for vid in gray_image_list:
#     #print(type(vid))
#     vid= np.array(vid)
#     standard_vid = standardize(vid)
#     # standardized_list.append(standard_vid)
#     float_vids= standard_vid.astype('float32')
#     standardized_list.append(float_vids)

In [169]:
# sample_names= [y for x,y in final]
# sample_names

In [161]:
# dict_keys=[]
# for name in sample_names:
#     dict_keys.append(name.replace('.mpg', ''))

In [168]:
# final_vids= dict(zip(dict_keys, standardized_list))


# final_vids

In [16]:
# np.savez(f'zipped_vids_s{a}.npz', **final_vids)

In [85]:
# data = np.load('zipped_vids.npz', allow_pickle=True).items()
# loaded_dict = {key: val for key, val in data}

# print(loaded_dict)
! pwd

/Users/alessiastroni/code/girishgautam/lip_translate/pipeline


In [19]:
# import json

# for key, value in final_vids.items():
#     if isinstance(value, np.ndarray):
#         final_vids[key] = value.tolist()

# with open ('preprocessed_data.json', 'w') as f:
#     json.dump(final_vids, f)

In [ ]:
def load_to_bucket(storage_filename=None, local_filename=None):
    storage_filename = "lip_reading/pipeline/zipped_vids.npz"
    local_filename = "zipped_vids.npz"

    client = storage.Client()
    bucket = client.bucket(lip_translate)
    blob = bucket.blob(storage_filename)
    blob.upload_from_filename(local_filename)


if __name__ =='__main__':
    load_to_bucket()
